<a href="https://colab.research.google.com/github/AshwinRS07/AMGC/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!fusermount -u drive
#!google-drive-ocamlfuse drive

In [ ]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=12c9a16acf2f208d88c91ae32d9e9f3ef5b03cad8e4d486c689a8dc009c54a2c
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [ ]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator
from sklearn.metrics import plot_confusion_matrix

In [ ]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [ ]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingset, instance, k):
    distances = []
    for x in range(len(trainingset)):
        dist = distance(trainingset[x], instance, k) + distance(instance,trainingset[x],k)
        distances.append((trainingset[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [ ]:
def nearestclass(neighbors):
    classVote = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
            
    sorter = sorted(classVote.items(), key=operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [ ]:
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return 1.0 * correct / len(testSet)

In [ ]:
%ls

drive/  sample_data/


In [ ]:
import librosa, IPython
import librosa.display
file = 'drive/MyDrive/AML Project/Data/genres_original/disco/disco.00031.wav'
signal, sr = librosa.load(file , sr = 22050) 
IPython.display.Audio(signal, rate=sr)

In [ ]:
directory = '/content/drive/MyDrive/AML Project/Data/genres_original'
%cd '/content/drive/MyDrive/AML Project/Data/genres_original/'
f = open('my.dat', "wb")
i = 0
for folder in os.listdir(directory):
    #print(folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+"/"+folder):
        #print(file)
        try:
            (rate, sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
f.close()

/content/drive/MyDrive/AML Project/Data/genres_original


In [ ]:
dataset = []

def loadDataset(filename, split, trset, teset):
    with open('my.dat','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trset.append(dataset[x])
        else:
            teset.append(dataset[x])

trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

In [ ]:
# Make the prediction using KNN(K nearest Neighbors)
length = len(testSet)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(trainingSet, testSet[x], 3)))

accuracy1 = getAccuracy(testSet, predictions)
print("Accuracy:", accuracy1)

Accuracy: 0.6882352941176471


In [ ]:
from collections import defaultdict
results = defaultdict(int)

directory = '/content/drive/MyDrive/AML Project/Data/genres_original'

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
pred = nearestclass(getNeighbors(dataset, testSet[2], 5))
print('Prediction:',results[pred])
print(testSet)

Streaming output truncated to the last 5000 lines.
         -0.53749128],
       [-19.33929451,  58.97807641,  30.60253434, -15.62284977,
         10.21041786,  -3.43965525,   3.55778335, -28.43077964,
        -20.10636987, -24.48717726,  -7.71784077, -27.38882228,
        -32.39009037],
       [-19.70845516,  30.60253434,  54.67492316, -16.71137784,
          9.15965349,  15.78964091,   4.82787164,   8.1202139 ,
        -18.66340674, -27.97077842, -17.60597356, -13.22418826,
        -26.25419763],
       [  5.60383656, -15.62284977, -16.71137784,  60.56574522,
          9.81847728, -10.60957267, -13.34513266, -15.80910154,
         15.65534204,  25.33989641,   7.8409106 ,   7.12038419,
         -4.27577032],
       [ -8.11983711,  10.21041786,   9.15965349,   9.81847728,
         49.95398715,   8.90094771,  -0.29430828, -11.61192471,
         -4.86621768, -16.88575997,  -9.80265662,  -3.20733515,
         -8.28869207],
       [ -7.90497036,  -3.43965525,  15.78964091, -10.60957267,
  

In [ ]:
plot_confusion_matrix(neighbours, X_test, y_test) 